In [ ]:
from fastai.vision.all import *

In [ ]:
path = Path('../input/cassava-leaf-disease-classification')

In [ ]:
train_df = pd.read_csv(path/'train.csv')
train_df = train_df[~train_df['image_id'].isin(['1562043567.jpg', '3551135685.jpg', '2252529694.jpg'])]

In [ ]:
def get_x(r):
    return path/'train_images'/r['image_id']

def get_y(r):
    return r['label']

In [ ]:
item_tfms = RandomResizedCrop(460, min_scale=0.75, ratio=(1.,1.))
batch_tfms = [*aug_transforms(size=384, max_warp=0), Normalize.from_stats(*imagenet_stats)]
bs=16

pets = DataBlock(blocks=(ImageBlock, CategoryBlock),
                 get_x=get_x,
                 get_y=get_y,
                 splitter=RandomSplitter(0.2, seed=42),
                 item_tfms=item_tfms,
                 batch_tfms=batch_tfms)
dls = pets.dataloaders(train_df, bs=bs)

In [ ]:
loss_func = LabelSmoothingCrossEntropy()

In [ ]:
learn = cnn_learner(dls, resnet50, metrics= [accuracy], loss_func=loss_func, opt_func=ranger, pretrained=False).to_native_fp16()

In [ ]:
learn.load('/kaggle/input/leaf-resnet50/model_6')

In [ ]:
learn = learn.to_native_fp32()

In [ ]:
submission_df = pd.read_csv(path/'sample_submission.csv')
submission_df.head()

In [ ]:
test_data_path = submission_df['image_id'].apply(lambda x: path/'test_images'/x)
tst_dl = learn.dls.test_dl(test_data_path)
predictions = learn.tta(dl = tst_dl, n=10)

submission_df['label'] = np.argmax(predictions[0],axis=1)
submission_df

In [ ]:
submission_df.to_csv('submission.csv',index=False)